# Задание 2. Итерационные методы решения линейных систем

In [10]:
from latexifier import latexify
from IPython.display import display, Markdown
import numpy as np
from typing import Callable, List, Tuple, Union
import warnings
warnings.filterwarnings('ignore')

In [17]:
class Matrix(np.ndarray):
    """extends np.ndarray"""

    def __new__(cls, input_array):
        obj = np.asarray(input_array).view(cls)
        return obj

    def inverse_rows(self, i: int, j: int):
        """Swaps i-th and j-th rows of a matrix"""
        self[i], self[j] = self[j].copy(), self[i].copy()

    def inverse_cols(self, i: int, j: int):
        """Swaps i-th and j-th columns of a matrix"""
        self = self.transpose()
        self.inverse_rows(i, j)
        self = self.transpose()
    
    def tex(self) -> str:
        """Latexifies the matrix rounding up values up to 6 decimal values"""
        return latexify(self)

    def norm(self) -> float:
        """Calculates Matrix norm using np.linalg.norm"""
        return np.linalg.norm(self, 1)
    def inv(self) -> Matrix:
        return Matrix(np.linalg.inv(self))

In [12]:
def Gauss(A: Matrix, B: Matrix, n: int) -> Matrix:
    """
    Вычисляет решения линейного уравнения Ax = B методом Гаусса с выбором
    главного элемента по столбцам (с перестановкой строк)
    """
    AB = np.append(A, B)
    X = [[0] for _ in range(n)]
    for k in range(n):
        j = k
        for i in range(k+1, n):
            if abs(AB[j][k]) < abs(AB[i][k]):
                j = i
        AB.inverse_rows(k, j)
        for j in range(k+1, n): 
            c = AB[j][k] / AB[k][k]
            for i in range(n+1):
                AB[j][i] -= c * AB[k][i]
    X[n-1][0] = AB[n-1][n] / AB[n-1][n-1]
    for k in range(n-1, -1, -1):
        s = 0
        for i in range(k+1, n):
            s += AB[k][i] * X[i][0]
        X[k][0] = (AB[k][n] - s) / AB[k][k]
    return Matrix(np.array(X))

In [13]:
def append(M: np.ndarray, N: np.ndarray) -> Matrix:
    """Appends two numpy arrays with axis=1"""
    return Matrix(np.append(arr=np.asarray(M), values=N, axis=1))


def print_tex(*argv) -> None:
    """Displays a LaTex Markdown output"""
    res = ' '.join(['$$'] + [(latexify(arg) if isinstance(arg, np.ndarray) else str(arg)) for arg in argv] + ['$$'])
    display(Markdown(res))


def get_latex_column(arg: str, n: int) -> str:
    """
    Prepares a string with LaTex syntax for a column of `arg` with indeces form 1 to `n`
    """
    res = [r"""\begin{pmatrix}""", "\n"]
    for i in range(n):
        line = arg + "_{" + str(i+1) + "}" + (r"\\" if i != n-1 else "") + "\n"
        res.append(line)
    res.append(r"""\end{pmatrix}""")
    return ''.join(res)


def get_exact_column(col: Matrix, n: int) -> str:
    """Prepares a string with LaTex syntax for a column of {col} elements"""
    res = [r"""\begin{pmatrix}""", "\n"]
    for i in range(n):
        line = str(float(col[i][0])) + (r"\\" if i != n-1 else "") + "\n"
        res.append(line)
    res.append(r"""\end{pmatrix}""")
    return ''.join(res)


def print_system_output(A: Matrix, s: str, B: Matrix, X: Matrix, n: int) -> None:
    """Oupputs the result of solving a linear equation"""
    B_ = np.matmul(A, X)
    print_tex('X^* = ', get_latex_column("x^*", n), '=', X.tex())
    print_tex(rf'{s} \times X^* = ', A.tex(), X.tex(), '=', B_.tex(), '= B^*')
    print_tex('B = ', get_exact_column(B, n), r'\stackrel{?}{=}', get_exact_column(B_, n), '= B^*')
    print_tex(r'B - B^* = ', get_exact_column(B - B_, n))

In [36]:
n = 4
A = Matrix(np.random.rand(n, n) * 10)
A_inv = A.inv()

In [37]:
print_tex(A, r'\times', A_inv, '=', A @ A_inv)
print(A.norm() * A_inv.norm())

$$ \begin{pmatrix} 1.671032 & 4.823063 & 6.282260 & 0.183900 \\ 0.829223 & 0.361655 & 4.481442 & 3.570561 \\ 4.792397 & 5.700438 & 3.674593 & 4.618789 \\ 8.566745 & 6.600569 & 9.144552 & 5.596271 \end{pmatrix} \times \begin{pmatrix} -0.142811 & -0.181202 & -0.157279 & 0.250111 \\ 0.139283 & -0.077212 & 0.242111 & -0.155137 \\ 0.093096 & 0.101548 & -0.149899 & 0.055868 \\ -0.097787 & 0.202517 & 0.200143 & -0.112492 \end{pmatrix} = \begin{pmatrix} 1 & 0 & 0 & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 1 & 0 \\ 0 & 0 & 0 & 1 \end{pmatrix} $$

17.67376478450909
